In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
!nvidia-smi

Wed Jan 25 23:08:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
def save_model():
    # Save the weights
    model.save_weights('./content/drive/MyDrive/HackathonData/model1/my_checkpoint')


def load_model(label_count_):
    model_ = create_model(label_count_)
    model_.load_weights('./content/drive/MyDrive/HackathonData/model1/my_checkpoint')
    return model_

In [ ]:
def eval_model_adv():
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print('Test Accuracy: %.3f' % acc)

    neg_y_test = []
    neg_X_test = []

    cad_y_test = []
    cad_X_test = []

    ckd_y_test = []
    ckd_X_test = []

    ibd_y_test = []
    ibd_X_test = []

    t2d_y_test = []
    t2d_X_test = []

    for idx, row in enumerate(y_test):
      if(y_test[idx][0] == 1):
        neg_y_test.append(y_test[idx])
        neg_X_test.append(X_test[idx])
      elif(y_test[idx][3] == 1):
        cad_y_test.append(y_test[idx])
        cad_X_test.append(X_test[idx])
      elif(y_test[idx][4] == 1):
        ckd_y_test.append(y_test[idx])
        ckd_X_test.append(X_test[idx])
      elif(y_test[idx][2] == 1):
        ibd_y_test.append(y_test[idx])
        ibd_X_test.append(X_test[idx])
      elif(y_test[idx][1] == 1):
        t2d_y_test.append(y_test[idx])
        t2d_X_test.append(X_test[idx])

    neg_y_test = np.array(neg_y_test)
    neg_X_test = np.array(neg_X_test)
    loss, acc = model.evaluate(neg_X_test, neg_y_test, verbose=1)
    print('Test Accuracy (only negatives): %.3f' % acc)

    pos_y_test = np.array(cad_y_test)
    pos_y_test = np.concatenate((pos_y_test, ckd_y_test))
    pos_y_test = np.concatenate((pos_y_test, ibd_y_test))
    pos_y_test = np.concatenate((pos_y_test, t2d_y_test))
    pos_X_test = np.array(cad_X_test)
    pos_X_test = np.concatenate((pos_X_test, ckd_X_test))
    pos_X_test = np.concatenate((pos_X_test, ibd_X_test))
    pos_X_test = np.concatenate((pos_X_test, t2d_X_test))
    loss, acc = model.evaluate(pos_X_test, pos_y_test, verbose=1)
    print('Test Accuracy (only positives): %.3f' % acc)

    cad_y_test = np.array(cad_y_test)
    cad_X_test = np.array(cad_X_test)
    loss, acc = model.evaluate(cad_X_test, cad_y_test, verbose=1)
    print('Test Accuracy (only cad): %.3f' % acc)

    ckd_y_test = np.array(ckd_y_test)
    ckd_X_test = np.array(ckd_X_test)
    loss, acc = model.evaluate(ckd_X_test, ckd_y_test, verbose=1)
    print('Test Accuracy (only ckd): %.3f' % acc)

    ibd_y_test = np.array(ibd_y_test)
    ibd_X_test = np.array(ibd_X_test)
    loss, acc = model.evaluate(ibd_X_test, ibd_y_test, verbose=1)
    print('Test Accuracy (only ibd): %.3f' % acc)

    t2d_y_test = np.array(t2d_y_test)
    t2d_X_test = np.array(t2d_X_test)
    loss, acc = model.evaluate(t2d_X_test, t2d_y_test, verbose=1)
    print('Test Accuracy (only t2d): %.3f' % acc)

In [ ]:
def load_data():
    # load the dataset
    dfs = [pd.read_csv('/content/drive/MyDrive/HackathonData/SpeciesT2D_train.csv'),
           pd.read_csv('/content/drive/MyDrive/HackathonData/SpeciesIBD_train.csv'),
           pd.read_csv('/content/drive/MyDrive/HackathonData/SpeciesCAD_train.csv'),
           pd.read_csv('/content/drive/MyDrive/HackathonData/SpeciesCKD_train.csv')]
    df_ = pd.concat([x if x['label'].replace(1, idx + 1, inplace=True) == True else x for idx, x in enumerate(dfs)], join="outer", axis=0).fillna(0).drop_duplicates()
    # split into input and output columns
    inputvalues = df_.drop(['label', 'sample_ID'], axis=1)
    outputvalues = df_['label']
    # outputvalues = y_vec
    X, y = inputvalues.values, outputvalues.values

    # example of random oversampling to balance the class distribution
    from collections import Counter
    from imblearn.over_sampling import RandomOverSampler
    # define oversampling strategy
    oversample = RandomOverSampler(sampling_strategy='minority')
    # fit and apply the transform
    X, y = oversample.fit_resample(X, y)
    # summarize class distribution
    print(Counter(y))


    # ensure all data are floating point values
    X = X.astype('float32')
    y = y.reshape(-1, 1)
    y = OneHotEncoder(categories=[['0', '1', '2', '3', '4']], sparse=False).fit_transform(y)

    # split into train and test datasets
    x_train_, x_test_, y_train_, y_test_ = train_test_split(X, y, test_size=0.3, random_state=1234) # seed
    x_train_, x_val_, y_train_, y_val_ = train_test_split(x_train_, y_train_, test_size=0.15, random_state=1234)
    print(x_train_.shape, x_test_.shape, y_train_.shape, y_test_.shape)
    # determine the number of input features
    n_features_ = x_train_.shape[1]
    return df_, x_train_, x_val_, x_test_, y_train_, y_val_, y_test_, n_features_, 5


def create_model(label_count_):
    # define model
    activation = 'relu'
    initializer = tf.keras.initializers.he_normal(seed=1234)
    model_ = Sequential()
    #model_.add(Dense(12000, activation=activation, kernel_initializer=initializer, input_shape=(n_features,)))
    #model_.add(Dense(14000, activation='relu', kernel_initializer=initializer))
    #model_.add(Dense(12000, activation='relu', kernel_initializer=initializer))
    model_.add(Dense(10000, activation=activation, kernel_initializer=initializer))
    model_.add(Dense(7000, activation=activation, kernel_initializer=initializer))
    model_.add(Dense(3000, activation=activation, kernel_initializer=initializer))
    model_.add(Dense(1000, activation=activation, kernel_initializer=initializer))
    model_.add(Dense(200, activation=activation, kernel_initializer=initializer))
    model_.add(Dense(50, activation=activation, kernel_initializer=initializer))
    model_.add(Dense(10, activation=activation, kernel_initializer=initializer))
    model_.add(Dense(label_count_, activation='sigmoid'))
    # compile the model
    model_.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model_


df, X_train, x_val, X_test, y_train, y_val, y_test, n_features, label_count = load_data()
model = create_model(label_count_=label_count)

Counter({0: 622, 4: 622, 1: 516, 2: 82, 3: 41})
(1120, 16901) (565, 16901) (1120, 5) (565, 5)


In [ ]:
def fit_model():
    # fit the model
    #callbacks_list = [PlotLearning()]
    model.fit(X_train, y_train, epochs=70, batch_size=1120, verbose=1) #callbacks=callbacks_list


fit_model()
#ave_model()
#model = load_model(5)
eval_model_adv()

Epoch 1/70
1/1 [==============================] - 1s 1s/step - loss: 17.4211 - accuracy: 0.3357
Epoch 2/70
1/1 [==============================] - 0s 427ms/step - loss: 3213.3049 - accuracy: 0.0116
Epoch 3/70
1/1 [==============================] - 0s 447ms/step - loss: 127.4238 - accuracy: 0.0116
Epoch 4/70
1/1 [==============================] - 0s 418ms/step - loss: 152.2791 - accuracy: 0.3339
Epoch 5/70
1/1 [==============================] - 0s 423ms/step - loss: 85.3971 - accuracy: 0.3205
Epoch 6/70
1/1 [==============================] - 0s 427ms/step - loss: 92.4487 - accuracy: 0.0455
Epoch 7/70
1/1 [==============================] - 0s 417ms/step - loss: 75.7571 - accuracy: 0.3339
Epoch 8/70
1/1 [==============================] - 0s 415ms/step - loss: 61.7555 - accuracy: 0.3339
Epoch 9/70
1/1 [==============================] - 0s 424ms/step - loss: 38.1376 - accuracy: 0.3339
Epoch 10/70
1/1 [==============================] - 0s 423ms/step - loss: 22.8897 - accuracy: 0.3339
Epoch 11